# SHIFT-23. Answers to questions on data

## 9. Impact of buying own products

In [1]:
import pandas as pd

In [2]:
own_products = pd.read_csv('products.csv').query('is_own_trademark == 1')[['product_id']]
own_products

,product_id
10,0012a164c0
23,001c8e3b3a
33,002cd4c65d
70,0067d95477
95,009997f0a4
...,...
42957,ff78e11173
42966,ff81efc13d
42980,ffa1f745ae
42994,ffba05c8fd


Clients who bought own products

In [3]:
train_purch_own = pd.read_csv('train_purch.csv')\
    .merge(own_products, on='product_id')

In [10]:
client_purch_own = train_purch_own[['client_id']].drop_duplicates()
client_purch_own

,client_id
0,000012768d
1,000ca87889
2,006cf006f4
3,00946ede17
5,00a346bdf8
...,...
2435383,bed6107c0f
2435771,6e1c9e5cf1
2435819,1dfbfd1444
2436761,96cd3b59f9


In [12]:
train = pd.read_csv('train.csv')\
    .merge(client_purch_own,
          on='client_id',
          how='left',
          indicator=True)
train

,client_id,treatment_flg,purchased,_merge
0,ad6561e2d8,1,1,both
1,7c1ccbf93f,1,1,both
2,b58fadcab6,1,1,both
3,e99e6fabb9,0,0,both
4,27fb6f8520,1,1,both
...,...,...,...,...
140022,999d284453,1,1,both
140023,f634deea4e,0,1,both
140024,16cb4f99b0,0,1,both
140025,23c2b72b2e,1,1,both


In [13]:
train._merge.unique()

['both', 'left_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [23]:
group_a_treat = train.loc[(train.treatment_flg == 1) & (train._merge == 'both'), 'purchased']\
    .value_counts(normalize=True)
group_a_treat

1    0.643697
0    0.356303
Name: purchased, dtype: float64

In [25]:
group_a_control = train.loc[(train.treatment_flg == 0) & (train._merge == 'both'), 'purchased']\
    .value_counts(normalize=True)
group_a_control

1    0.61195
0    0.38805
Name: purchased, dtype: float64

In [26]:
uplift_a = group_a_treat[1] - group_a_control[1]
uplift_a

0.03174733889332593

In [27]:
group_b_treat = train.loc[(train.treatment_flg == 1) & (train._merge == 'left_only'), 'purchased']\
    .value_counts(normalize=True)
group_b_treat

0    0.598392
1    0.401608
Name: purchased, dtype: float64

In [28]:
group_b_control = train.loc[(train.treatment_flg == 0) & (train._merge == 'left_only'), 'purchased']\
    .value_counts(normalize=True)
group_b_control

0    0.677419
1    0.322581
Name: purchased, dtype: float64

In [29]:
uplift_b = group_b_treat[1] - group_b_control[1]
uplift_b

0.07902768028809476

In [30]:
uplift_a - uplift_b

-0.04728034139476883